In [ ]:
import matplotlib.pyplot as plt
from battery_model import BatteryModel
from soc_estimation import SOCEstimator
from soh_estimation import SOHEstimator
from thermal_model import ThermalModel
from monitor import monitor

import numpy as np
import os

class BatteryModel:
    def __init__(self, capacity_ah=2.3, ocv=3.7, resistance=0.05):
        self.capacity = capacity_ah
        self.ocv = ocv
        self.resistance = resistance

    def get_terminal_voltage(self, current):
        return self.ocv - current * self.resistance

class SOCEstimator:
    def __init__(self, capacity_ah, initial_soc=1.0):
        self.capacity = capacity_ah
        self.soc = initial_soc

    def update(self, current, dt):
        self.soc -= (current * dt) / (self.capacity * 3600)
        self.soc = max(0, min(1.0, self.soc))
        return self.soc

class ThermalModel:
    def __init__(self, mass=0.5, cp=900, ambient_temp=25.0):
        self.temperature = ambient_temp
        self.mass = mass
        self.cp = cp
        self.ambient = ambient_temp

    def update(self, current, resistance, dt, cooling_coeff=0.01):
        heat_gen = (current ** 2) * resistance
        cooling = cooling_coeff * (self.temperature - self.ambient)
        delta_temp = (heat_gen - cooling) * dt / (self.mass * self.cp)
        self.temperature += delta_temp
        return self.temperature


class SOHEstimator:
    def __init__(self, rated_capacity_ah):
        self.rated_capacity = rated_capacity_ah
        self.current_capacity = rated_capacity_ah

    def degrade(self, cycles, degradation_per_cycle=0.0005):
        self.current_capacity = max(0.5 * self.rated_capacity, self.rated_capacity * (1 - degradation_per_cycle * cycles))
        return self.current_capacity / self.rated_capacity



def monitor(voltage, temperature, soc):
    alerts = []
    if voltage > 4.2:
        alerts.append("Overvoltage!")
    if voltage < 3.0:
        alerts.append("Undervoltage!")
    if temperature > 60:
        alerts.append("Overtemperature!")
    if temperature < 0:
        alerts.append("Undertemperature!")
    if soc < 0.1:
        alerts.append("Low SOC!")
    return alerts


os.makedirs("results", exist_ok=True)

battery = BatteryModel()
soc_estimator = SOCEstimator(battery.capacity)
soh_estimator = SOHEstimator(battery.capacity)
thermal_model = ThermalModel()

duration = 3600  # seconds (1 hour)
dt = 1
time = np.arange(0, duration, dt)
current_profile = np.random.normal(1.0, 0.2, len(time))  # 1A average

soc_history = []
soh_history = []
temp_history = []
voltage_history = []
alerts_list = []

for i, t in enumerate(time):
    current = current_profile[i]
    voltage = battery.get_terminal_voltage(current)
    soc = soc_estimator.update(current, dt)
    temp = thermal_model.update(current, battery.resistance, dt)
    soh = soh_estimator.degrade(i // 3600)

    soc_history.append(soc)
    soh_history.append(soh)
    temp_history.append(temp)
    voltage_history.append(voltage)
    alerts_list.append(monitor(voltage, temp, soc))

plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
plt.plot(time, soc_history)
plt.title("State of Charge (SOC)")
plt.grid()

plt.subplot(2, 2, 2)
plt.plot(time, soh_history)
plt.title("State of Health (SOH)")
plt.grid()

plt.subplot(2, 2, 3)
plt.plot(time, temp_history)
plt.title("Temperature (°C)")
plt.grid()

plt.subplot(2, 2, 4)
plt.plot(time, voltage_history)
plt.title("Battery Voltage (V)")
plt.grid()

plt.tight_layout()
plt.savefig("results/simulation_results.png")
plt.show()
